In [31]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)


In [125]:
COLS_DIAGS = ["Capítulo", "Sección", "Categoría", "Descripción"]

In [126]:
df_procesada = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")
df_procesada["FECHA 1º evaluación"] = pd.to_datetime(
    df_procesada["FECHA 1º evaluación"], format="%Y-%m-%d"
)
df_procesada["F NAC"] = pd.to_datetime(df_procesada["F NAC"], yearfirst=True)


# Analisis para Congenitas

En este analisis se quieren responder obtener los siguientes insights:

- Frecuencia de diagnosticos por anio
- Frecuencia de edad de primera consulta por anio
- Frecuencia de procedencia por anio
- Frecuencia de sexo por anio
- Frecuencia de grupo etario por anio

## Frecuencia de diagnosticos por anio

In [127]:
egresos_por_anio = (
    df_procesada.groupby(["ANIO_PRIMERA_EVALUACION", "MES_PRIMERA_EVALUACION"])[
        "DIAGNOSTICO PRINCIPAL", "validacion"
    ]
    .value_counts()
    .reset_index(name="conteo")
)

## Frecuencia de edad de primera consulta por anio

In [132]:
edad_primera_consulta = (
    df_procesada["FECHA 1º evaluación"] - df_procesada["F NAC"]
) / np.timedelta64(1, "Y")

df_procesada["EDAD_PRIMERA_CONSULTA"] = edad_primera_consulta
edades_validas = df_procesada.query("EDAD_PRIMERA_CONSULTA >= 0")
edades_validas["RANGO_ETARIO_PRIMERA_CONSULTA"] = pd.cut(
    edades_validas["EDAD_PRIMERA_CONSULTA"], [0, 15, 20, 25, 40, np.inf]
)


edades_por_diags_por_anio = (
    edades_validas.groupby(
        ["ANIO_PRIMERA_EVALUACION", "RANGO_ETARIO_PRIMERA_CONSULTA"]
    )["validacion"]
    .value_counts()
    .reset_index(name="conteo")
)